In [5]:
import pandas as pd
import numpy as np
from preprocess import scale
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
'''
import nb    # naive bayes classification
import lda   # 
import rda   # 
import qda   # quadratic linear regression
import lr    # logistic regression
'''
xxx = 0

In [6]:
url = 'https://raw.githubusercontent.com/probml/pmtkdata/master/spamData/spam.data'
data = pd.read_table(url, header = None, sep = ' ')

In [7]:
y = data[57].values
X = data[range(57)].values
del data

In [8]:
train = np.random.choice(len(y), replace = False, size = 3065)
test = np.ones(len(y), dtype=bool)
test[train] = 0

In [9]:
train_y = y[train]
test_y = y[test]
train_X = X[train,]
test_X = X[test,]
del X
del y

In [12]:
import numpy as np
from scipy import stats

stats.bernoulli.logpmf(3, 0.5)

class NaiveBayes:
    '''
    murphy pp. 84 to 89
    binary features only (this implementation)
    '''    
    def fit(self, X, y):        
        N, D = X.shape
        cs = np.unique(y)
        priors = []
        thetas = []
        
        for c in cs:
            i = y == c
            X_c = X[i]
            priors.append(np.mean(i))
            thetas.append(np.mean(X_c, axis = 0))
            
        self.cs = cs
        self.priors = np.array(priors)
        self.thetas = np.log(thetas)
        
    def predict(self, X):
        N, D = X.shape
        C = len(self.cs)
        eps = np.spacing(0)
        log_priors = np.log(self.priors)
        log_theta = np.log(self.thetas + eps)
        log_not_theta = np.log(1 - self.thetas + eps)
        not_X = 1 - X
        
        loglik = np.empty((N, C))
        l = stats.bernoulli.logpmf
        
        for c in range(C):
            loglik[:, c] = log_priors[c] + X.dot(l(X, log_theta[c, :])) + not_X.dot(l(X, log_not_theta[c, :]))

        return self.cs[np.argmax(loglik, axis = 1)]    
        
clf = NaiveBayes()
clf.fit(1 * (train_X > 0), train_y)
y_hat = clf.predict(test_X)
print '%0.3f' % np.mean(1.0 - np.mean(y_hat == test_y))

ValueError: shapes (1536,57) and (1536,57) not aligned: 57 (dim 1) != 1536 (dim 0)

In [165]:
def test(transform, C):
    clf = LogisticRegression(C = C)
    # clf = GaussianNB()
    clf.fit(transform(train_X), train_y)
    result = lambda X, y: '%0.3f' % np.mean(1.0 - np.mean(clf.predict(transform(X)) == y))
    print result(train_X, train_y), result(test_X, test_y)

for C in np.arange(0.5, 2, 0.5):
    print C
    test(lambda X: X, C)
    test(lambda X: scale(X), C)
    test(lambda X: np.log(X + 0.1), C)
    test(lambda X:  1 * (X > 0), C)

0.5
0.072 0.079
0.074 0.080
0.054 0.050
0.063 0.074
1.0
0.073 0.079
0.073 0.085
0.054 0.050
0.063 0.072
1.5
0.071 0.078
0.071 0.085
0.054 0.051
0.061 0.073
